#  Introduction

After scraping an e-commerce website in my last project, https://github.com/ASBIK-ABDELLAH/Avito_scraping, i've decided to make this data useful by cleaning it and by trying to build a machine learning model which predict the price of cars.

This project will contain my pre-processing of data and machine learning models.

## **The Dataset**

The dataset is a CSV file, which contains the following columns:
 
 'car_city' : The name of city where is the car sold.\
 'car_cv_man_auto': Horsepower of the car and being automatic or manual.\
 'car_equipements': Car equipements.\
 'car_information ': Some information about the car.\
 'car_name': The car name.\
 'car_price': The car price.

# 1. **Importing necessary module for Cleaning**

In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
df = pd.read_csv("../input/av-cars/items_av_cars_1.csv")
df

**I will create a bunch of functions based on my analysis of data, which will clean my CSV file.**

In [ ]:
##### Droping columns that don't contain the full information about if the car is manuel or automatic.
def drop_cv_manuelle(df,column):
    df["{}".format(column)] = df["{}".format(column)].apply(
        lambda x: np.nan if (str(x)[-1] =="-") or (str(x)[0] =="-") or ("CV" not in str(x)) else x)

In [ ]:
def clean_price(df,column):
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: str(x)[:len(x)-3]) ## delete ,DH
    def space(x):
        while str(x).find(" ") != -1 :
            x = str(x)[:str(x).find(" ")] + str(x)[str(x).find(" ")+1:]                 ### delete space between numbers.
        return x
    df["{}".format(column)] = df["{}".format(column)].apply(
        lambda x: space(x) )

    df["{}".format(column)] = df["{}".format(column)].astype(float)
    df.where(df["{}".format(column)] > 10000, inplace=True)

In [ ]:
def clean_city(df,column):
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: str(x)[:str(x).find(",")]) ## find "," and choose city

In [ ]:
def clean_car_cv_man_auto(df,column):
    df["fuel"] = df["{}".format(column)].apply(lambda x: str(x)[:str(x).find(",")])
    df["CV"] = df["{}".format(column)].apply(lambda x: str(x)[str(x).find(",")+1:str(x).rfind(",")])
    df["CV"] = df["CV"].apply(lambda x: str(x)[0] if "P" not in str(x) else "41")
    df["man_auto"] = df["{}".format(column)].apply(lambda x: str(x)[str(x).rfind(",")+1:])

In [ ]:
def clean_car_information(df,column):
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: str(x)[16:])
    df["{}".format(column)] = df["{}".format(column)].str.split(",")
    
    def alphabet_up(x):
        for i in string.ascii_uppercase:
            x = x [1:] if i in x[0] else x
        return x
    def alphabet_low(x):
        for i in string.ascii_lowercase:
            x = x [1:] if i in x[0] else x
        return x
    
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: alphabet_up(x))
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: alphabet_low(x))
  
    df["Kilometrage"] = df["{}".format(column)].apply(lambda x: x[0] if '-' in x[0] else(x[1] if ('-' in x[1]) and ('0' or '9') in x[1] else np.nan))
    df.dropna(subset=["Kilometrage"], axis='index', how='any', inplace=True)
    
    

    df["First_selling"] = df["{}".format(column)].apply(lambda x: True if "Oui" in str(x) else False)
    
    df["Year_of_Model"] = df["{}".format(column)].apply(lambda x: x[1] if '-'and ' ' not in x[1] else(x[0] if ('-' and ' ' not in x[0]) else "1980"))
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: x[2:-1] if "Oui" or "Non" in x[-1] else x[2:])
    
    
    df["Origin_car"] = df["{}".format(column)].apply(
        lambda x: x[-1] if ('WW au Maroc'== x[-1] or 'Dédouanée'== x[-1] or 'Importée neuve'== x[-1] or 'Pas encore dédouanée'== x[-1])  else np.nan)
    df["{}".format(column)] = df["{}".format(column)].apply(
        lambda x: x[:-1] if ('WW au Maroc'== x[-1] or 'Dédouanée'== x[-1] or 'Importée neuve'== x[-1] or 'Pas encore dédouanée'== x[-1]) else x)
    
    
    
    df["Mark"] = df["{}".format(column)].apply(lambda x: x[0])
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: x[1:] if len(x) >1 else np.nan)

   
    df["Model"] = df["{}".format(column)].apply(lambda x: x[0] if type(x) is list else np.nan)
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: x[1:] if type(x) is list else x)
    
    
    df["Nbr_doors"] = df["{}".format(column)].apply(lambda x: x[0] if type(x) is list and len(x) != 0 else np.nan )
    

In [ ]:
### convert equipements to binary
def clean_car_equipements(df,column):
    df["Jantes_aluminium"] = df["{}".format(column)].apply(
        lambda x: False if "Jantes aluminium" not in str(x) else True)
    df["Airbags"] = df["{}".format(column)].apply(
        lambda x: False if "Airbags" not in str(x) else True)
    df["Climatisation"] = df["{}".format(column)].apply(
        lambda x: False if "Climatisation" not in str(x) else True)
    df["navigation_Systeme"] = df["{}".format(column)].apply(
        lambda x: False if "Système de navigation" not in str(x) else True)
    df["GPS"] = df["{}".format(column)].apply(
        lambda x: False if "GPS" not in str(x) else True)
    df["Sièges_cuir"] = df["{}".format(column)].apply(
        lambda x: False if "Sièges cuir" not in str(x) else True)
    df["Radar_back"] = df["{}".format(column)].apply(
        lambda x: False if "Radar de recul" not in str(x) else True)
    df["Back_camera"] = df["{}".format(column)].apply(
        lambda x: False if "Caméra de recul" not in str(x) else True)
    df["Electric_windows"] = df["{}".format(column)].apply(
        lambda x: False if "Vitres électriques" not in str(x) else True)
    df["ABS"] = df["{}".format(column)].apply(
        lambda x: False if "ABS" not in str(x) else True)
    df["ESP"] = df["{}".format(column)].apply(
        lambda x: False if "ESP" not in str(x) else True)
    df["Speed_regulator"] = df["{}".format(column)].apply(
        lambda x: False if "Régulateur de vitesse" not in str(x) else True)
    df["Vitess_limitor"] = df["{}".format(column)].apply(
        lambda x: False if "Limiteur de vitesse" not in str(x) else True)
    df["CD"] = df["{}".format(column)].apply(
        lambda x: False if "CD" not in str(x) else True)
    df["MP3"] = df["{}".format(column)].apply(
        lambda x: False if "MP3" not in str(x) else True)
    df["Bluetooth"] = df["{}".format(column)].apply(
        lambda x: False if "Bluetooth" not in str(x) else True)
    df["Board_computer"] = df["{}".format(column)].apply(
        lambda x: False if "Ordinateur de bord" not in str(x) else True)
    df["Remote_central_locking"] = df["{}".format(column)].apply(
        lambda x: False if "Verrouillage centralisé à distance" not in str(x) else True)

In [ ]:
## Some values in Kilometrage columns contained words isntead of number, so i replace them by a nan value.
def clean_Kilometrage(df,column):
    def space(x):
        while str(x).find(" ") != -1 :
            x = str(x)[:str(x).find(" ")] + str(x)[str(x).find(" ")+1:]
        return x
    df["{}".format(column)] = df["{}".format(column)].apply(
        lambda x: space(x))
    
    def alphabet_up(x):
        for i in string.ascii_uppercase:
            x = str(x) if i not in str(x) else np.nan
        return x
    def alphabet_low(x):
        for i in string.ascii_lowercase:
            x = str(x) if i not in str(x) else np.nan
        return x
    
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: alphabet_up(x))
    df["{}".format(column)] = df["{}".format(column)].apply(lambda x: alphabet_low(x))
    df.dropna(subset=["{}".format(column)], axis='index', how='any', inplace=True)
    df["{}".format(column)] = df["{}".format(column)].str.split("-")
    
    df["{}".format(column)] = df["{}".format(column)].apply(
         lambda x: (float(x[0]) + float(x[1]))/2 if len(x) !=1 else float(x[0]))

In [ ]:
pd.set_option('display.max_colwidth', None)

df = df.drop(['_type'], axis=1)
df = df.dropna(subset=['car_price', 'car_information'], axis='index', how='any')

drop_cv_manuelle(df, "car_cv_man_auto")
clean_price(df, "car_price")
clean_city(df, "car_city")
df = df.dropna(subset=['car_cv_man_auto'], axis='index', how='any')
clean_car_cv_man_auto(df, "car_cv_man_auto")
df.drop(['car_cv_man_auto'], axis=1, inplace=True)
clean_car_information(df, 'car_information')
df.drop(['car_information'], axis=1, inplace=True)
clean_car_equipements(df, "car_equipements")
df.drop(['car_equipements'], axis=1, inplace=True)
clean_Kilometrage(df,'Kilometrage')
df = df.where(df['car_price'] >= 10000.0)
df.drop(['car_name'], axis=1, inplace=True)
df.dropna(subset=['Nbr_doors'], axis='index', how='any', inplace=True)

df["Year_of_Model"] = df["Year_of_Model"].astype(int)
df["Nbr_doors"] = df["Nbr_doors"].astype(int)
df['CV'] = df['CV'].astype(int)

df.dropna(subset=['Mark'], axis='index', how='any', inplace=True)
df.dropna(subset=['Origin_car'], axis='index', how='any', inplace=True)
df = df[df["Mark"] != "Autres"]


In [ ]:
df

In [ ]:
y0 = df['car_price'].values
y0 = y0.reshape(-1, 1)
y0.shape

In [ ]:
import matplotlib.pyplot as plt
'''for i in df.columns:
    x1 = df['{}'.format(i)]
    plt.scatter(x1, y0, color='blue')
    plt.xlabel(i)
    plt.show()
'''

In [ ]:
##### After visualisation, i made some intervals smaller because I've noticed that some data are misleading the learning of the algorithms.
df = df.where(df['car_price'] < 1.75 * 10**6)
df = df.dropna(subset=['car_price'], axis='index', how='any')
df = df.where(df['CV'] > 4)
df = df.where(df['CV'] < 40)
df = df.dropna(subset=['CV'], axis='index', how='any')

In [ ]:
####Convert qualitative values to binary using dummies in the following columns.#### 
dummies_man_auto = pd.get_dummies(df.man_auto)
dummies_fuel = pd.get_dummies(df.fuel)
dummies_Origin_car = pd.get_dummies(df.Origin_car)
dummies_Mark = pd.get_dummies(df.Mark)
dummies_Model = pd.get_dummies(df.Model)

In [ ]:
df = pd.concat([df,dummies_man_auto,dummies_fuel,dummies_Origin_car,dummies_Mark,dummies_Model], axis=1)  #
df.drop(['man_auto','car_city','fuel','Origin_car','Mark','Model'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df['lancer'].dtype

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X = df.drop(['car_price'], axis=1).values

In [ ]:
from sklearn.preprocessing import scale
X_scaled = scale(X)

In [ ]:
y = df['car_price'].values
y = y.reshape(-1, 1)
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state=123)

In [ ]:
import xgboost
'''
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.5,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

model.fit(X_train, y_train)
model.score(X_test, y_test)
'''



In [ ]:
#from sklearn.preprocessing import StandardScaler
#from sklearn.decomposition import PCA
#scaler = StandardScaler()
#X_train1 = scaler.fit_transform(X_train1)
#X_test1 = scaler.fit_transform(X_test1)

model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.5,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

model.fit(X_train, y_train)
model.score(X_test, y_test)